In [1]:
import time

# --- Import Gremlin-Python modules ---
import gremlin_python
import gremlin_python.driver.serializer as se
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.client import Client
from gremlin_python.driver.serializer import GraphSONSerializersV2d0
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Cardinality
from gremlin_python.process.traversal import Column
from gremlin_python.process.traversal import Direction
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import Scope
from gremlin_python.process.traversal import Barrier
from gremlin_python.process.traversal import Bindings
from gremlin_python.process.traversal import WithOptions
# ---------------------------------------

In [2]:
# Function: check response message from gremlin server
# return: (flag, result_list).
#          if successed, flag == True and result_list holds the message/data if available.
#          if failed, flag == False and result_list holds the raised error message.
def CheckResult(result):
    if result is not None:
        i = 0
        while result.done.running():
            time.sleep(0.1)
            i +=1
#             print (i)
            if i>300:
                break

        a = result.all()
        s = str(a)
        ss= s.lower()
        if 'finished' in ss:
            if 'error' in ss:
                return False, [s]
            else:
                return True, a.result()
        else:
            return False, [s]
    else:
        return False, []

In [ ]:
# ****************************************************
#  Connect to gremlin server
# ****************************************************

In [3]:
# --- Create client to do low-level operations with gremlin server ---
client = Client("ws://192.168.1.52:8182/gremlin",    # URL of gremlin server (janusgraph application interface)
                "g",                                 # name of traversal object
                username="",                         # username to access Janusgraph
                password="")                         # password to access Janusgraph
print (client)

In [4]:
res = client.submit('Gremlin.version()')
f,x = CheckResult(res)
print (f)
print ('Version of gremlin server is ',x)

True
Version of gremlin server is  ['3.3.3']


In [ ]:
# ****************************************************
#  Create a new graph property with Janusgraph
# ****************************************************

In [ ]:
# --- Create graph on the remote gremlin server: ---
# res = client.submit("graph = TinkerGraph.open()")                  # with Gremlin Server
# res = client.submit("graph = JanusGraphFactory.open('inmemory')")  # with JanusGraph
# CheckResult(res)     # Ignore the 'rasised GremlinServerError' message and go ahead

In [ ]:
# --- launch an operation of loading graphml data file on remote server: ---
# res = client.submit("graph.io(graphml()).readGraph('c:/datatools/~test/air-routes.graphml')")
# CheckResult(res)

In [ ]:
# ****************************************************
#  Check/Open exitsting graph properties in Janusgraph
# ****************************************************

In [5]:
#  Checking existings graph properties - ALL
res = client.submit('JanusGraphFactory.getGraphNames()')
CheckResult(res)

(True, ['graph', 'ConfigurationManagementGraph', 'airroutes'])

In [6]:
#  Checking existings graph properties
res = client.submit('ConfiguredGraphFactory.getGraphNames()')
CheckResult(res)

(True, ['airroutes'])

In [7]:
# --- Open the selected existing graph property on the remote Janusgraph server: ---
res = client.submit("graph = ConfiguredGraphFactory.open('airroutes')")  # 'airrouts' is a already created graph
CheckResult(res)       # Ignore the 'rasised GremlinServerError' message and go ahead

(False, ['<Future at 0x1b7b2cd4550 state=finished raised GremlinServerError>'])

In [ ]:
# **************************************************************
#  Test client.submit approach which simulate a gremlin console
# **************************************************************

In [8]:
# <Test-1> --- Check graph's features ---
res = client.submit("graph.features()")
f,r = CheckResult(res)
if f:
    print (r[0])

FEATURES
> GraphFeatures
>-- Transactions: true
>-- ThreadedTransactions: true
>-- ConcurrentAccess: true
>-- Persistence: true
>-- Computer: true
> VariableFeatures
>-- Variables: true
>-- IntegerValues: true
>-- UniformListValues: false
>-- FloatValues: true
>-- LongArrayValues: true
>-- StringValues: true
>-- ByteValues: true
>-- DoubleArrayValues: true
>-- StringArrayValues: true
>-- MixedListValues: false
>-- SerializableValues: false
>-- LongValues: true
>-- BooleanArrayValues: true
>-- MapValues: true
>-- DoubleValues: true
>-- ByteArrayValues: true
>-- BooleanValues: true
>-- FloatArrayValues: true
>-- IntegerArrayValues: true
> VertexFeatures
>-- RemoveVertices: true
>-- MultiProperties: true
>-- DuplicateMultiProperties: true
>-- AddVertices: true
>-- MetaProperties: true
>-- CustomIds: false
>-- RemoveProperty: true
>-- NumericIds: true
>-- StringIds: false
>-- AddProperty: true
>-- UserSuppliedIds: false
>-- AnyIds: false
>-- UuidIds: false
> VertexPropertyFeatures
>-- Cust

In [9]:
# <Test-2> --- Create graph traversal object g ---
res = client.submit("g = graph.traversal()")
CheckResult(res)    # Ignore the 'rasised GremlinServerError' message and go ahead

(False, ['<Future at 0x1b7b2cf6eb8 state=finished raised GremlinServerError>'])

In [10]:
# <Test-3> --- Optional sample code using client method ---
#
# WARNING: 
#    The following codes only work with the 'air-routes.graphml' data file!
#

r =  client.submit("g.V().count()").all().result()
print (len(r))
print (r)

1
[32573]


In [11]:
r1 = client.submit("g.V().has('city','Sydney')").all().result()
print (type(r1))
print (r1)
print (len(r1),'vertices')
print (r1[0].id, r1[0].label)

<class 'list'>
[v[9003032], v[17182744], v[17661976], v[35536920], v[36048920], v[8876088], v[10387512], v[3280984], v[4141144], v[7176280], v[12951640], v[45192], v[2510984], v[9797768], v[12644488], v[4608176], v[9576624], v[11047088], v[7319744], v[7975104], v[8700096], v[12124352], v[540872], v[1523912], v[1921224], v[2752712], v[10363080]]
27 vertices
9003032 airport


In [12]:
r2 = client.submit("g.V().has('city','Sydney').has('country','AU')").all().result()
print (len(r2))
print (r2)

18
[v[9003032], v[17182744], v[17661976], v[35536920], v[36048920], v[8876088], v[10387512], v[4141144], v[7176280], v[12951640], v[45192], v[2510984], v[4608176], v[7319744], v[8700096], v[1523912], v[2752712], v[10363080]]


In [13]:
r3 = client.submit("g.V().range(0,10)").all().result()
print (len(r3))
print (r3)

10
[v[4120], v[8216], v[12312], v[16408], v[20504], v[24600], v[28696], v[32792], v[36888], v[40984]]


In [14]:
r4 = client.submit("g.V().range(100,120)").all().result()
print (len(r4))
print (r4)

20
[v[413720], v[417816], v[421912], v[426008], v[430104], v[434200], v[438296], v[442392], v[446488], v[450584], v[454680], v[458776], v[462872], v[466968], v[471064], v[475160], v[479256], v[483352], v[487448], v[491544]]


In [ ]:
# **************************************************************
#  Test python as a gremlin language variant
# **************************************************************

In [15]:
# Create graph and remote graph traversal objects (g) by RemoteConnection method:
# Take the most advantages out of Gremlin-Python

# statics.load_statics(globals())

graph = Graph()
print ('type of graph: ',type(graph))

# Binding local traversal object g with the remote traversal object g:
g = graph.traversal().withRemote(
        DriverRemoteConnection('ws://192.168.1.52:8182/gremlin', # URL of gremlin server (janusgraph application interface)
                               'g',                              # name of traversal object
                               username="",                      # username to access Janusgraph
                               password="")                      # password to access Janusgraph 
        )
print ('type of g: ',type(g))

type of graph:  <class 'gremlin_python.structure.graph.Graph'>
type of g:  <class 'gremlin_python.process.graph_traversal.GraphTraversalSource'>


In [16]:
x1 = g.V().has("city",'Sydney').toList()
print (len(x1))
print (x1)

27
[v[9003032], v[17182744], v[17661976], v[35536920], v[36048920], v[8876088], v[10387512], v[3280984], v[4141144], v[7176280], v[12951640], v[45192], v[2510984], v[9797768], v[12644488], v[4608176], v[9576624], v[11047088], v[7319744], v[7975104], v[8700096], v[12124352], v[540872], v[1523912], v[1921224], v[2752712], v[10363080]]


In [17]:
x2 = g.V().both()[1:3].toList()
print (len(x2))
print (x2)

2
[v[4232], v[4272]]


In [18]:
x3 = g.V().range(1,10).toList()
print (len(x3))
print (x3)

9
[v[8216], v[12312], v[16408], v[20504], v[24600], v[28696], v[32792], v[36888], v[40984]]


In [19]:
x4 = g.V(x3).toList()
print (len(x4))
print (x4)

9
[v[8216], v[12312], v[16408], v[20504], v[24600], v[28696], v[32792], v[36888], v[40984]]


In [20]:
x5 = g.V(x3).id().toList()
print (len(x5))
print (x5)

9
[8216, 12312, 16408, 20504, 24600, 28696, 32792, 36888, 40984]


In [21]:
g.V().range(0,10).toList()

[v[4120],
 v[8216],
 v[12312],
 v[16408],
 v[20504],
 v[24600],
 v[28696],
 v[32792],
 v[36888],
 v[40984]]

In [22]:
# Done and close the client
client.close()